# Imports & Dataset Loading

In [1]:
pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 torch

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import pandas as pd
import ast

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Creating the directories that will be used during execution

**Important note here:** The "data" folder will hold the data needed to fine-tune the model. You need to provide this data to the model. In my experiments I generated most of it using the Alpaca method, as described in the repo. The results folder is just there for the model to have somewhere to push its predictions.

In [2]:
current_dir = os.getcwd()

folders = ["data", "results"]

for folder in folders:
    folder_path = os.path.join(current_dir, folder)
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

## Dataset preprocessing and save

In [3]:
df = pd.read_csv("data/CORRECTED_PROMPTS_FOR_FINE_TUNING.csv").rename(columns={"sql_query": "prompt", "tables": "response"})
df['prompt'] = "##EXTRACTTABLES:\n" + df['prompt']

train_df = df[["prompt", "response"]].sample(frac=0.9, random_state=101)
test_df  = df[["prompt", "response"]].drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('data/train.jsonl', orient='records', lines=True)
test_df.to_json('data/test.jsonl', orient='records', lines=True)

## Visualizing examples:

In [4]:
train_df.head()

,prompt,response
37,##EXTRACTTABLES:\nWITH cte_enrollment AS (\n ...,"['education.institution.enrollment', 'educatio..."
109,##EXTRACTTABLES:\nWITH cte1 AS (\n SELECT p.p...,"['patients', 'appointments', 'doctors', 'presc..."
31,##EXTRACTTABLES:\nWITH cte1 AS (\n SELECT \n ...,"['insurance.customers', 'insurance.policies', ..."
89,##EXTRACTTABLES:\nWITH cte1 AS (\n SELECT\n ...,"['news.articles', 'news.categories', 'news.com..."
66,##EXTRACTTABLES:\nWITH cte1 AS (\n SELECT \...,"['customers', 'orders', 'products', 'order_pro..."


In [5]:
position = 0

In [6]:
# Prompt at position

print(train_df["prompt"].iloc[position])

##EXTRACTTABLES:
WITH cte_enrollment AS (
  SELECT
    student_id,
    COUNT(DISTINCT course_id) AS num_courses
  FROM
    education.institution.enrollment
  GROUP BY
    student_id
  HAVING
    COUNT(DISTINCT course_id) >= 3
),
cte_average_grade AS (
  SELECT
    student_id,
    AVG(grade) AS avg_grade
  FROM
    education.institution.grades
  GROUP BY
    student_id
),
cte_top_students AS (
  SELECT
    e.student_id,
    e.num_courses,
    g.avg_grade
  FROM
    cte_enrollment e
    JOIN cte_average_grade g ON e.student_id = g.student_id
  WHERE
    g.avg_grade >= 80
),
cte_course_stats AS (
  SELECT
    c.course_id,
    COUNT(DISTINCT e.student_id) AS num_students,
    SUM(g.grade) AS total_grade
  FROM
    education.institution.courses c
    LEFT JOIN education.institution.enrollment e ON c.course_id = e.course_id
    LEFT JOIN education.institution.grades g ON e.student_id = g.student_id AND c.course_id = g.course_id
  GROUP BY
    c.course_id
),
cte_top_courses AS (
  SELECT
    

In [7]:
# Outputs at position

my_list = ast.literal_eval(train_df["response"].iloc[position])
for table in my_list:
  print(table)

education.institution.enrollment
education.institution.grades
education.institution.courses
education.institution.students


# Hyperparameters for LLAMA 2 model

In [4]:
# Set model and training data paths
model_name = "NousResearch/llama-2-7b-chat-hf"
dataset_name = "data/train.jsonl"
new_model = "llama-2-7b-sql-parser"

# LoRA Configs
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

# Model hyperparameters
output_dir = "results"
num_train_epochs = 3
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

# Loading Dataset

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [15]:
system_message = "You are a SQL code reference table finder. You receive a SQL query and return all references to tables"

# Load datasets
train_dataset = load_dataset('json', data_files='data/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='data/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

### Bits and Bytes config

In [16]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

### HF Model Config

In [17]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 2/2 [01:54<00:00, 57.19s/it]


### PEFT Config

In [18]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

### Setting up Training Arguments

In [19]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)

### Creating Trainer Instance

In [20]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 20/20 [00:00<00:00, 1219.93 examples/s]


## Training...

In [21]:
trainer.train()
trainer.model.save_pretrained(new_model)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
5,0.663500,0.739448
10,0.865400,0.583755
15,0.493700,0.503857
20,0.476900,0.411998
25,0.341900,0.348327
30,0.287200,0.304760
35,0.285300,0.284120
40,0.250400,0.267132
45,0.230900,0.253380
50,0.190400,0.247205


### Loosely Evaluating

In [22]:
# Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n##EXTRACTTABLES:\nSELECT * FROM [database].[table1].new_tensors WHERE datatensor_length > 1000 [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=600)
result = pipe(prompt)
print(result[0]['generated_text'])

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[INST] <<SYS>>
You are a SQL code reference table finder. You receive a SQL query and return all references to tables
<</SYS>>

##EXTRACTTABLES:
SELECT * FROM [database].[table1].new_tensors WHERE datatensor_length > 1000 [/INST] ['database', 'table1', 'new_tensors']


# Inference

#### Inference on a known result

In [24]:
# Set index to run inference on:
index = 0



test_text = test_df.iloc[index]["prompt"]
expected_response = test_df.iloc[index]["response"]

prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{test_text} [/INST]" # replace the command here with something relevant to your task
num_new_tokens = 100  # change to the number of new tokens you want to generate

# Count the number of tokens in the prompt
num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

# Calculate the maximum length for the generation
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline(
    'text-generation', 
    model=model, 
    tokenizer=tokenizer, 
    max_length=max_length,
    do_sample=True,
    temperature=0.2
)

result = gen(prompt)
print("PROMPT:")
print(prompt)
print("\n\n")
print("RESULT:")
print(result[index]['generated_text'].replace(prompt, ''))

print("Expected Response:")
for table in ast.literal_eval(expected_response):
  print(table)

PROMPT:
[INST] <<SYS>>
You are a SQL code reference table finder. You receive a SQL query and return all references to tables
<</SYS>>

##EXTRACTTABLES:
WITH CustomerDeliveredOrders AS (
  SELECT 
    CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
    o.order_id,
    o.order_date,
    o.order_status,
    o.total_price
  FROM 
    [FreightCoDB].[OrderManagement].shipment_orders o
  JOIN 
    [FreightCoDB].[ClientRelations].clients c ON o.client_id = c.client_id
  WHERE 
    o.order_status = 'Delivered'
),

OrderProducts AS (
  SELECT 
    p.item_name,
    p.item_category,
    p.item_price,
    oi.order_id,
    oi.quantity
  FROM 
    [FreightCoDB].[OrderManagement].order_contents oi
  JOIN 
    [FreightCoDB].[Inventory].cargo_items p ON oi.item_id = p.item_id
),

CustomerProductDetails AS (
  SELECT 
    cdo.customer_name,
    cdo.order_id,
    cdo.order_date,
    cdo.total_price,
    op.item_name,
    op.item_category,
    op.item_price,
    op.quantity,
    op.item_price * o

#### Inference on a created query:

In [25]:
query = """##EXTRACTTABLES:
SELECT repo.CodeText, 
repo.Lang, 
MAX(repo.DateEdit) as FinalEdit, 
repo.Name, 
repo.Repository, 
repo.Branch, 
scr.Length
FROM [targetcurves].[repodata].repositories repo
LEFT JOIN [targetcurves].[info].scripts scr
ON repo.Name = scr.ScriptName
WHERE scr.Length < 1000
"""

query_base = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{query} [/INST]"

expected_result = """["[targetcurves].[repodata].repositories", "[targetcurves].[info].scripts"]"""

result = gen(query)

print(result)
print(expected_result)

[{'generated_text': "##EXTRACTTABLES:\nSELECT repo.CodeText, \nrepo.Lang, \nMAX(repo.DateEdit) as FinalEdit, \nrepo.Name, \nrepo.Repository, \nrepo.Branch, \nscr.Length\nFROM [targetcurves].[repodata].repositories repo\nLEFT JOIN [targetcurves].[info].scripts scr\nON repo.Name = scr.ScriptName\nWHERE scr.Length < 1000\nGROUP BY repo.CodeText, \nrepo.Lang, \nrepo.Name, \nrepo.Repository, \nrepo.Branch\nHAVING COUNT(DISTINCT scr.ScriptName) > 1\nORDER BY repo.Name ASC; [/INST] [INST] <<SYS>>\nYou are a SQL code reference table finder. You receive a SQL query and return all references to tables\n<</SYS>>\n\nYou receive the following SQL query:\n\nSELECT \n    repo.CodeText, \n    repo.Lang, \n    MAX(repo.DateEdit) as FinalEdit, \n    repo.Name, \n    repo.Repository, \n    repo.Branch, \n    scr.Length\nFROM \n    [targetcurves].[repodata].repositories repo\nLEFT JOIN \n    [targetcurves].[info].scripts scr\nON \n    repo.Name = scr.ScriptName\nWHERE \n    scr.Length < 1000\nGROUP BY \n 

# Merging the Model with the LoRA weights:

In [5]:
model_path = f"{new_model}"  # change to your preferred path

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Loading checkpoint shards: 100%|██████████| 2/2 [03:36<00:00, 108.21s/it]


('llama-2-7b-sql-parser/tokenizer_config.json',
 'llama-2-7b-sql-parser/special_tokens_map.json',
 'llama-2-7b-sql-parser/tokenizer.json')